# DVS Records of Measurements

A record of datasets recorded for DVS tests on XXX, along with assessment of completeness for each category of test.

NB: this excludes datasets recorded for set-to-work activities.


                                                                                   Last updated 14/08/2025

In [ ]:
%matplotlib inline
import pylab as plt
import numpy as np
import time
import katpoint

In [ ]:
from dvs import util, planning, tipcurve, driftscan, hologreport
from analysis import katselib
import dvsholog

In [ ]:
ant = "s0XXX"
final_FI_angles = {'x':103, 's':-18.7, 'b2':-100}

In [ ]:
def discard(bad_cbids):
    """ Discard datasets from **global** lists 'rr'and 'cbids', and also from local cache. """
    global rr, cbids
    for cbid in bad_cbids:
        rr = [r for r in rr if (int(r['CaptureBlockId'])!=cbid)] # Remove from 'rr'
        try: # In case it's been cached locally, remove it
            cbids.remove(cbid); util.open_dataset(cbid, cache_root="./l1_data").del_cache()
        except (NameError, ValueError): # If 'cbids' not yet defined, or 'cbid' not in 'cbids'
            pass

def summarise_distribution(cbids, ant, match_FI_angle=None, DFI=0.02):
    """ Summarises the counts of datasets over elevation and by night & day """
    sub_cbids = list(cbids)
    el, is_night = [], []
    for cbid in list(cbids):
        ds = util.open_dataset(cbid, ref_ant=ant, cache_root="./l1_data")
        is_nighttime = katselib.is_nighttime(ds.timestamps[0]) and katselib.is_nighttime(np.mean(ds.timestamps))
        fi = katselib.getsensorvalues("%s_dsm_indexerActualPosition"%ant, ds.timestamps)[1]
        if (match_FI_angle is not None) and (np.abs(np.median(fi)-match_FI_angle)>DFI):
            sub_cbids.remove(cbid)
        else:
            el.append(np.mean(ds.el))
            is_night.append(is_nighttime)
        print(cbid, {False:"day", True:"night"}[is_nighttime], np.mean(ds.el), "degEl; FI @", np.mean(fi))
    plt.figure(figsize=(12,3))
    plt.hist(el, bins=20, range=(14,84), label="day & night")
    plt.hist(np.compress(is_night, el, axis=0), bins=20, range=(14,84), alpha=0.5, label="night")
    plt.xlabel("Elevation [deg]"); plt.ylabel("Counts [#]"); plt.legend()
    print(sub_cbids)
    return sub_cbids

# Tipping Curves
* Pick time for Sun<0degEl, MilkyWay<10degEl
* Pick Az angles to avoid local hills & Milky way - unique per DISH

In [ ]:
def qa_tipping(rr, ant):
    cbids = []
    for r in rr:
        cbids.append(int(r['CaptureBlockId']))
        try:
            ds = util.open_dataset(cbids[-1], ref_ant=ant, cache_root="./l1_data")
            ds.select(ants=ant, scans="track")
            ra, dec, el = np.transpose([(np.mean(ds.ra), np.mean(ds.dec), np.mean(ds.el)) for scan in ds.scans()])
            T_skytemp = tipcurve.Sky_temp(np.min(ds.freqs)/1e6)
            fig = T_skytemp.plot_sky(ra, dec, pointstyle='ro', date=time.gmtime(ds.timestamps[0])[:6],
                                     **dict(zip(['lat','lon','elev'], map(float, ds.ants[0].position_wgs84))))
            sun_ra, sun_dec = np.array(katpoint.Target("Sun, special").radec(ds.timestamps[-1], ds.ants[0]))*180/np.pi
            T_skytemp.plot_sky(sun_ra, sun_dec, pointstyle='y*', fig=fig)
            plt.title("%d: %s - %s UTC, tipping at Az %.fdeg\nSky brightness at %.fMHz" % 
                      (cbids[-1], np.datetime64(int(ds.timestamps[0]),'s'), np.datetime64(int(ds.timestamps[-1]),'s'), np.mean(ds.az), T_skytemp.nu))
        except:
            util.open_dataset(cbids[-1], ref_ant=ant, cache_root="./l1_data").del_cache()
            cbids.pop(-1)
    return cbids

## Band2

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND CenterFrequency:1284000000 AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:*tipping*",
                         fields=["CaptureBlockId", "StartTime", "CenterFrequency", "Description", "Antennas"])

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_tipping(rr, ant)

In [ ]:
print(cbids)

## S0

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND CenterFrequency:2187500000 AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:*tipping*",
                         fields=["CaptureBlockId", "StartTime", "CenterFrequency", "Description", "Antennas"])

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_tipping(rr, ant)

In [ ]:
print(cbids)

## S4

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND CenterFrequency:3062500000 AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:*tipping*",
                         fields=["CaptureBlockId", "StartTime", "CenterFrequency", "Description", "Antennas"])

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_tipping(rr, ant)

In [ ]:
print(cbids)

# Driftscans
These should span the full range from below 20degEl to above 75degEl

In [ ]:
def qa_driftscans(rr, ant):
    cbids = []
    for r in rr:
        cbids.append(int(r['CaptureBlockId']))
        try:
            util.open_dataset(cbids[-1], cache_root="./l1_data")
            driftscan.DriftDataset(cbids[-1], ant=ant, debug=True)
        except:
            util.open_dataset(cbids[-1], cache_root="./l1_data").del_cache()
            cbids.pop(-1)
    return cbids

## Band2

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND CenterFrequency:1284000000 AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:drift*",
                         fields=["CaptureBlockId", "StartTime", "CenterFrequency", "Description", "Antennas"])

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_driftscans(rr, ant)

In [ ]:
print("Final FI angle = ", final_FI_angles['b2']) # For reference
summarise_distribution(cbids, ant);

## S0

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND CenterFrequency:2187500000 AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:drift*",
                         fields=["CaptureBlockId", "StartTime", "CenterFrequency", "Description", "Antennas"])

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_driftscans(rr, ant)

In [ ]:
print("Final FI angle = ", final_FI_angles['s']) # For reference
summarise_distribution(cbids, ant);

## S4

In [ ]:
rr = katselib.ls_archive(f"Antennas:*{ant}* AND CenterFrequency:3062500000 AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:drift*",
                         fields=["CaptureBlockId", "StartTime", "CenterFrequency", "Description", "Antennas"])

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_driftscans(rr, ant)

In [ ]:
print("Final FI angle = ", final_FI_angles['s']) # For reference
summarise_distribution(cbids, ant);

## Catch-up Planning

In [ ]:
schedule = [("->S0", 0),
            ("Hercules A", 15*60),("Hercules A", 15*60),("J0408-6545", 35*60),
            ("J0408-6545", 35*60),("J0408-6545", 35*60),
            ("Pictor A", 22*60),("J0408-6545", 35*60),("Pictor A", 22*60),

            ("-> S4",10*60),
            ("PKS 1934-63", 28*60),("PKS 1934-63", 28*60),
            ("Tipping curves",60*60),
            ("PKS 1934-63", 28*60),("PKS 1934-63", 28*60),
]

Tobs = katpoint.Timestamp("2025-07-19 23:15:00").secs # UTC
planning.sim_observations(schedule, catfn="../catalogues/fluxmodels.csv", Tstart=Tobs, el_limit_deg=15,
                 figsize=(12,7))

# Holography

In [ ]:
cached_url = lambda cbid: f"./l1_data/{cbid}/{cbid}_sdp_l0.full.rdb"
xyzoffsets = [0.0, 1.5363, -3.4570]

beacon_freq = {"EUTELSAT 21B": [12500,12500.5],
            "EUTELSAT 10B": [11698.6,11699.2],
            "EUTELSAT 8 WEST B": 12501,
            "EUTELSAT 36D": [11701.8,12501],
            "INTELSAT 37E (IS-37E)": 12501,
            "INTELSAT 22 (IS-22)": [11699,12499],
            "INTELSAT 25 (IS-25)": 12254,
            "INTELSAT 17 (IS-17)": 12502,
            "INTELSAT 10-02": 11452,
            "BADR-7": [11703,11703.5],
            "THOR 7": [11706,12494.5,12495.5],
            "YAMAL 402": [11459.5,12500],
            "APSTAR-7": [11699.8,12251],
           }

def qa_holography(rr, ant, f_MHz, dMHz=1):
    cbids = []
    if isinstance(f_MHz, dict): # Map of satellite beacons: check up-front if there are any missing
        for r in rr:
            tgts = sorted(set([_ for _ in r['Targets'] if not "unwrap" in _]))
            if (tgts[-1] not in f_MHz.keys()):
                print("MISSING: no beacon defined for %s"%tgts)
        
    for r in rr:
        freqs = f_MHz
        if isinstance(f_MHz, dict):
            tgt = sorted(set([_ for _ in r['Targets'] if not "unwrap" in _]))[-1]
            try:
                freqs = f_MHz[tgt]
            except: # If it's missing, don't process this record
                continue
        
        cbids.append(int(r['CaptureBlockId']))
        try:
            util.open_dataset(cbids[-1], cache_root="./l1_data")
            ds = dvsholog.Dataset(cached_url(cbids[-1]), "SKA", scanantname=ant, method='gainrawabs')
            beams = []
            for f in np.atleast_1d(freqs):
                try: # This approach is necessary in case we combine different frequency bands
                    beams.append(dvsholog.BeamCube(ds, scanantennaname=ant, freqMHz=f, dMHz=dMHz, interpmethod='scipy', xyzoffsets=xyzoffsets))
                except:
                    pass
            assert len(beams)>0, "No beams loaded successfully"
            plt.figure(figsize=(14,3), layout='constrained'); plt.suptitle(cbids[-1])
            for i,bm in enumerate(beams):
                plt.subplot(1,len(beams),i+1); bm.plot("Gx", doclf=False); plt.gca().images[-1].colorbar.remove(); plt.title(bm.freqgrid)
        except:
            util.open_dataset(cbids[-1], cache_root="./l1_data").del_cache()
            cbids.pop(-1)
    return cbids

## High Resolution
These should typically be around 50degEl (optimal alignment, and "median" to match drift scan datasets)

## Band2

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:\"scan-extent 24*\" AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_holography(rr, ant, f_MHz=[900, 1150, 1600, 1705], dMHz=10) # Middle 2 typically have RFI - judge how bad?

In [ ]:
print("Final FI angle = ", final_FI_angles['b2']) # For reference
summarise_distribution(cbids, ant);

## S0

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND InstructionSet:\"scan-extent 13*\" AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_holography(rr, ant, f_MHz=[1780, 2100, 2250, 2450], dMHz=15) # Middle 2 typically have RFI - judge how bad?

In [ ]:
print("Final FI angle = ", final_FI_angles['s']) # For reference
summarise_distribution(cbids, ant);

## S4

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND CenterFrequency:3062500000 AND InstructionSet:\"scan-extent 9*\" AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_holography(rr, ant, f_MHz=[2650, 2910, 3160, 3410], dMHz=28) # Typically have RFI - judge how bad?

In [ ]:
print("Final FI angle = ", final_FI_angles['s']) # For reference
summarise_distribution(cbids, ant);

## Ku-band

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND CenterFrequency:[11000000000 TO *] AND InstructionSet:*holography*" +\
                         " AND InstructionSet:\"scan-extent 6.0\"",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=250)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
         # Stick to "broken" & "sparse" only?
])

In [ ]:
cbids = qa_holography(rr, ant, beacon_freq, dMHz=1)

In [ ]:
print("Final FI angle = ", final_FI_angles['x']) # For reference
summarise_distribution(cbids, ant);

## Over Elevation

## Band2

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND CenterFrequency:1284000000 AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_holography(rr, ant, f_MHz=[900, 1150, 1600, 1705], dMHz=10) # Middle 2 typically have RFI - judge how bad?

In [ ]:
print("Final FI angle = ", final_FI_angles['b2']) # For reference
summarise_distribution(cbids, ant, match_FI_angle=final_FI_angle, DFI=0.08); # 1753730647 is reported inaccurately

## S0

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND CenterFrequency:2187500000 AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_holography(rr, ant, f_MHz=[1800, 2000, 2200, 2500], dMHz=20) # Typically have RFI - judge how bad?

In [ ]:
print("Final FI angle = ", final_FI_angles['s']) # For reference
summarise_distribution(cbids, ant, match_FI_angle=final_FI_angles['b2']);

## S3 & S4

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND CenterFrequency:[2843750000 TO 3062500000] AND InstructionSet:*holography*",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=240)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = qa_holography(rr, ant, f_MHz=[2650, 2910, 3160], dMHz=28) # Typically have RFI - judge how bad?

In [ ]:
print("Final FI angle = ", final_FI_angles['s']) # For reference
summarise_distribution(cbids, ant, match_FI_angle=final_FI_angles['s']);

## Ku-band

In [ ]:
rr = katselib.ls_archive(f"InstructionSet:\"scan-ant*{ant}*\" AND StartTime:[2025-05-20T0:0:0Z TO *] AND CenterFrequency:[11000000000 TO *] AND InstructionSet:*holography*" +\
                         " AND NOT InstructionSet:\"scan-extent 6.0\"",
                    min_duration=1800, fields=["CaptureBlockId","StartTime","CenterFrequency",'Description','InstructionSet'], field_len=250)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
         # Stick to "broken" & "sparse" only?
])

In [ ]:
f_holog = [11496,11696,11896,12070] + [12700,12872,13072,13272] # X0 + X3
cbids = qa_holography(rr, ant, f_MHz=f_holog, dMHz=110)

In [ ]:
print("Final FI angle = ", final_FI_angles['x']) # For reference
summarise_distribution(cbids, ant, match_FI_angle=final_FI_angles['x']);

## Catch-up Planning

In [ ]:
# Schedule for 27/07/2025
schedule = [("Band2",0),
            ("PKS 1934-63", 1*60*60),
            ("Band2->S4",25*60),
            ("PKS 1934-63", 2*.75*60*60), ("PKS 1934-63", 2*.75*60*60), # Holography @ 50degEl
            ("J1924-2914", 2*.75*60*60), # Holography > 70degEl
            ("PKS 0408-65", 2*.75*60*60), ("3C 48", 2*.75*60*60), ("3C 48", 2*.75*60*60), ("3C 48", 2*.75*60*60)# Holography @ 20degEl
]

Tobs = katpoint.Timestamp("2025-07-27 17:30:00").secs # UTC
planning.sim_observations(schedule, catfn="../catalogues/targets_pnt_S.csv", Tstart=Tobs, el_limit_deg=14,
                 figsize=(12,7))

# All Sky Pointing

In [ ]:
def summarise_pointing(rr, ant, f_MHz, match_FI_angle=None, DFI=0.02):
    cbids = []
    for r in rr:
        if (float(r['CenterFrequency'])/1e6 != f_MHz):
            continue
        cbids.append(int(r['CaptureBlockId']))
        try:
            ds = util.open_dataset(cbids[-1], ref_ant=ant, cache_root="./l1_data")
            is_nighttime = katselib.is_nighttime(ds.timestamps[0]) and katselib.is_nighttime(np.mean(ds.timestamps))
            fi = katselib.getsensorvalues("%s_dsm_indexerActualPosition"%ant, ds.timestamps)[1]
            if (match_FI_angle is not None) and (np.abs(np.median(fi)-match_FI_angle)>DFI):
                cbids.pop(-1)
            else:
                print(cbids[-1], {False:"day", True:"night"}[is_nighttime], np.mean(ds.el), "degEl; FI @", np.mean(fi))
                katselib.plot_pointing(ds, requested=False, ants=ant, scans="~slew", collate=True, figsize=(12,5))
        except:
            util.open_dataset(cbids[-1], cache_root="./l1_data").del_cache()
            cbids.pop(-1)
    print(cbids)
    return cbids

In [ ]:
# Tilt corrections were DISABLED by default	
rr = katselib.ls_archive(f"StartTime:[2025-04-26T10:0:0Z TO 2025-05-16T18:00:00Z] AND Description:All Sky Pointing AND InstructionSet:*circular_pointing*" +
                         f" AND InstructionSet:\"scan-ant {ant}\"~1", min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","InstructionSet","Antennas"], field_len=220)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = summarise_pointing(rr, ant, 1284)

In [ ]:
# S4
cbids = summarise_pointing(rr, ant, 3062.5)

In [ ]:
cbids = summarise_pointing(rr, ant, 13072)

In [ ]:
# From here onward, tilt corrections were ENABLED unless explicitly noted otherwise! 	
rr = katselib.ls_archive(f"StartTime:[2025-05-16T18:00:00Z TO *] AND Description:All Sky Pointing AND InstructionSet:*circular_pointing*" +
                          f" AND InstructionSet:\"scan-ant {ant}\"~1", min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","InstructionSet","Antennas"], field_len=220)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

rr1 = [r for r in rr if ("no-tiltcorrection" not in r['InstructionSet'])]
rr0 = [r for r in rr if ("no-tiltcorrection" in r['InstructionSet'])]

In [ ]:
# B2
cbids0 = summarise_pointing(rr0, ant, 1284)

In [ ]:
cbids1 = summarise_pointing(rr1, ant, 1284)

In [ ]:
# S3
cbids0 = summarise_pointing(rr0, ant, 2843.75)

In [ ]:
cbids1 = summarise_pointing(rr1, ant, 2843.75)

In [ ]:
#S4
cbids0 = summarise_pointing(rr0, ant, 3062.5)

In [ ]:
cbids1 = summarise_pointing(rr1, ant, 3062.5)

In [ ]:
# X3
cbids0 = summarise_pointing(rr0, ant, 13072)

In [ ]:
cbids1 = summarise_pointing(rr1, ant, 13072)

# Tracking Stability

In [ ]:
# All data was recorded after tilt corrections were ENABLED by default
rr = katselib.ls_archive(f"StartTime:[2025-04-26T10:0:0Z TO *] AND Description:Stability AND InstructionSet:*circular_pointing*" +
                         f" AND InstructionSet:\"scan-ant {ant}\"~1", min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","InstructionSet","Antennas"], field_len=220)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = summarise_pointing(rr, ant, 13072)

# Indexer Repeatability

In [ ]:
# All data was recorded after tilt corrections were ENABLED by default
rr = katselib.ls_archive(f"StartTime:[2025-04-26T10:0:0Z TO *] AND Description:Index Repeatability AND InstructionSet:*circular_pointing*" +
                         f" AND InstructionSet:\"scan-ant {ant}\"~1", min_duration=900, fields=["CaptureBlockId","StartTime","CenterFrequency","Description","InstructionSet","Antennas"], field_len=220)

In [ ]:
# Run this before and after the "QA" cell below, to exclude datasets based on manual QA assessment
discard([# Update this list to contain all "bad" datasets, with comment please!
])

In [ ]:
cbids = summarise_pointing(rr, ant, 3062.5)

In [ ]:
cbids = summarise_pointing(rr, ant, 12499.5)